In [ ]:
# default_exp modules

In [ ]:
#export

from abc import ABCMeta
from traitlets import HasTraits, Dict
from abc import ABCMeta, abstractmethod
from io import StringIO
from typing import Optional, Mapping, Any
import traittypes
from IPython.display import display
from ipywidgets import FileUpload, Output
import pandas as pd

class DharpaModule(metaclass=ABCMeta):

    def __init__(self, **config):

        self._config: Mapping[str, Any] = config
        self._outputs: Mapping[str, Any] = None

        self._values: HasTraits = self._get_values()
        self._outputs: Mapping[str, Any] = self._get_outputs()

    @property
    def values(self):
        return self._values

    def test(self):
        pass

    @property
    def outputs(self):
        return self._outputs

    @abstractmethod
    def _get_values(self) -> HasTraits:
        pass

    @abstractmethod
    def _get_outputs(self) -> Mapping[str, Any]:
        pass

class FileInputModule(DharpaModule):

    def _get_values(self) -> HasTraits:

        class InputFileValue(HasTraits):

            file_details = Dict()
            content_data = traittypes.DataFrame()

        return InputFileValue()

    def _get_outputs(self) -> Mapping[str, Any]:

        uploaded_file_details = Output()
        file_upload = FileUpload(accept=".csv", multiple=False)

        def upload_handler(change):

            uploaded_file = list(change.new.values())[0]
            data = uploaded_file["content"].decode("utf-8")
            df = pd.read_csv(StringIO(data), header='infer')

            uploaded_file_details.clear_output()
            with uploaded_file_details:
                print(uploaded_file["metadata"])
                display(df)

        file_upload.observe(upload_handler, names=['value'])

        return {
            "file_details": uploaded_file_details,
            "file_upload": file_upload
        }

In [ ]:
from nbdev.export import notebook2script; notebook2script()


Converted 00_core.ipynb.
Converted 01_modules.ipynb.
